In [8]:
from PIL import Image
import os, glob, sys, numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

img_dir = "../Plastic"
categories = ["recycle", "Non_recycle"]
np_classes = len(categories)

image_w = 28
image_h = 28


pixel = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    img_dir_detail = img_dir + "/" + cat
    files = glob.glob(img_dir_detail+"/*.jpg")


    for i, f in enumerate(files):
        try:
            img = Image.open(f)
            img = img.convert("RGB")
            img = img.resize((image_w, image_h))
            data = np.asarray(img)
            #Y는 0 아니면 1이니까 idx값으로 넣는다.
            X.append(data)
            y.append(idx)
            if i % 300 == 0:
                print(cat, " : ", f)
        except:
            print(cat, str(i)+" 번째에서 에러 ")
X = np.array(X)
Y = np.array(y)


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

xy = (X_train, X_test, Y_train, Y_test)
np.save("../Plastic/binary_image_data.npy", xy)

recycle  :  ../Plastic/recycle\recycle1.jpg
recycle  :  ../Plastic/recycle\recycle127.jpg
recycle  :  ../Plastic/recycle\recycle154.jpg
recycle  :  ../Plastic/recycle\recycle181.jpg
recycle  :  ../Plastic/recycle\recycle208.jpg
recycle  :  ../Plastic/recycle\recycle235.jpg
recycle  :  ../Plastic/recycle\recycle2620.jpg
recycle  :  ../Plastic/recycle\recycle2891.jpg
recycle  :  ../Plastic/recycle\recycle3160.jpg
recycle  :  ../Plastic/recycle\recycle3430.jpg
recycle  :  ../Plastic/recycle\recycle533.jpg
recycle  :  ../Plastic/recycle\recycle804.jpg
Non_recycle  :  ../Plastic/Non_recycle\Non-recycle1.jpg
Non_recycle  :  ../Plastic/Non_recycle\Non-recycle1278.jpg
Non_recycle  :  ../Plastic/Non_recycle\Non-recycle1548.jpg
Non_recycle  :  ../Plastic/Non_recycle\Non-recycle1818.jpg
Non_recycle  :  ../Plastic/Non_recycle\Non-recycle2088.jpg
Non_recycle  :  ../Plastic/Non_recycle\Non-recycle2358.jpg
Non_recycle  :  ../Plastic/Non_recycle\Non-recycle2628.jpg
Non_recycle  :  ../Plastic/Non_recyc

C:\Users\happy\anaconda3\envs\tensorflow\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K

X_train, X_test, y_train, y_test = np.load('../Plastic/binary_image_data.npy', allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])
print(np.bincount(y_train))
print(np.bincount(y_test))

(6048, 28, 28, 3)
6048
[3166 2882]
[350 323]


In [10]:
image_w = 28
image_h = 28
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255


with K.tf_ops.device('/device:GPU:0'):
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(32, (3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model_dir = './model'
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    model_path = model_dir + "./recycle_classify.model"
    
    checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=7)

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 32)        896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 3, 3, 64)         

In [12]:
history = model.fit(X_train, y_train, batch_size=64, epochs=100, validation_split=0.15, callbacks=[checkpoint, early_stopping])

Train on 5140 samples, validate on 908 samples
Epoch 1/100
5140/5140 [==============================] - 4s 863us/step - loss: 0.6726 - accuracy: 0.5574 - val_loss: 0.5104 - val_accuracy: 0.7445

Epoch 00001: val_loss improved from inf to 0.51045, saving model to ./model./recycle_classify.model
Epoch 2/100
5140/5140 [==============================] - 4s 739us/step - loss: 0.4538 - accuracy: 0.7790 - val_loss: 0.3794 - val_accuracy: 0.8304

Epoch 00002: val_loss improved from 0.51045 to 0.37939, saving model to ./model./recycle_classify.model
Epoch 3/100
5140/5140 [==============================] - 4s 781us/step - loss: 0.3784 - accuracy: 0.8315 - val_loss: 0.3520 - val_accuracy: 0.8381

Epoch 00003: val_loss improved from 0.37939 to 0.35197, saving model to ./model./recycle_classify.model
Epoch 4/100
5140/5140 [==============================] - 4s 749us/step - loss: 0.3172 - accuracy: 0.8628 - val_loss: 0.3007 - val_accuracy: 0.8623

Epoch 00004: val_loss improved from 0.35197 to 0.3006

In [13]:
print("정확도 : %.2f " %(model.evaluate(X_test, y_test)[1]))

673/673 [==============================] - 0s 320us/step
정확도 : 0.98 


In [15]:
from PIL import Image
import os, glob, numpy as np
from keras.models import load_model

import tensorflow as tf

seed = 5
tf.set_random_seed(seed)
np.random.seed(seed)

caltech_dir = '../Plastic/test'

image_w = 28
image_h = 28

pixels = image_h * image_w * 3

X = []
filenames = []
files = glob.glob(caltech_dir+"/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)

    filenames.append(f)
    X.append(data)

X = np.array(X)
X = X.astype(float) / 255
model = load_model('./model/recycle_classify.model')

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt = 0
for i in prediction:
    if i >= 0.5:
        print("해당 " + filenames[cnt].split("\\")[0] + filenames[cnt].split("\\")[1] + "  이미지는 재활용불가능 으로 추정됩니다.")
    else :
        print("해당 " + filenames[cnt].split("\\")[0] + filenames[cnt].split("\\")[1] + "  이미지는 재활용가능 으로 추정됩니다.")
    cnt += 1

해당 ../Plastic/testNon-recycle16.jpg  이미지는 재활용불가능 으로 추정됩니다.
해당 ../Plastic/testNon-recycle3.jpg  이미지는 재활용불가능 으로 추정됩니다.
해당 ../Plastic/testNon-recycle34.jpg  이미지는 재활용불가능 으로 추정됩니다.
해당 ../Plastic/testNon-recycle95.jpg  이미지는 재활용불가능 으로 추정됩니다.
해당 ../Plastic/testrecycle11.jpg  이미지는 재활용가능 으로 추정됩니다.
해당 ../Plastic/testrecycle24.jpg  이미지는 재활용가능 으로 추정됩니다.
해당 ../Plastic/testrecycle466.jpg  이미지는 재활용불가능 으로 추정됩니다.
해당 ../Plastic/testrecycle64.jpg  이미지는 재활용가능 으로 추정됩니다.
